# **Part 1: Take a Look at Cartpole Rl Agent**

#### 1. According to the tutorials, if we want to edit the environment configuration, action space, observation space, reward function, or termination condition of the Isaac-Cartpole-v0 task, which file should we look at, and where is each part located?

**Source code:** https://github.com/isaac-sim/IsaacLab/blob/main/source/extensions/omni.isaac.lab_tasks/omni/isaac/lab_tasks/manager_based/classic/cartpole/cartpole_env_cfg.py

We should look at cartpole_env_cfg.py that located in Home/IsaacLab/blob/main/source/extensions/omni.isaac.lab_tasks/omni/isaac/lab_tasks/manager_based/classic/cartpole/cartpole_env_cfg.py and each part located

**Environment configuration** in line 157-181


In [ ]:
@configclass
class CartpoleEnvCfg(ManagerBasedRLEnvCfg):
    """Configuration for the cartpole environment."""

    # Scene settings
    scene: CartpoleSceneCfg = CartpoleSceneCfg(num_envs=4096, env_spacing=4.0)
    # Basic settings
    observations: ObservationsCfg = ObservationsCfg()
    actions: ActionsCfg = ActionsCfg()
    events: EventCfg = EventCfg()
    # MDP settings
    rewards: RewardsCfg = RewardsCfg()
    terminations: TerminationsCfg = TerminationsCfg()

    # Post initialization
    def __post_init__(self) -> None:
        """Post initialization."""
        # general settings
        self.decimation = 2
        self.episode_length_s = 5
        # viewer settings
        self.viewer.eye = (8.0, 0.0, 5.0)
        # simulation settings
        self.sim.dt = 1 / 120
        self.sim.render_interval = self.decimation

**Action space** in line 58-62

In [ ]:
@configclass
class ActionsCfg:
    """Action specifications for the MDP."""

    joint_effort = mdp.JointEffortActionCfg(asset_name="robot", joint_names=["slider_to_cart"], scale=100.0)

**Observation space** in line 65-85

In [ ]:
@configclass
class ObservationsCfg:
    """Observation specifications for the MDP."""

    @configclass
    class PolicyCfg(ObsGroup):
        """Observations for policy group."""

        # observation terms (order preserved)
        joint_pos_rel = ObsTerm(func=mdp.joint_pos_rel)
        joint_vel_rel = ObsTerm(func=mdp.joint_vel_rel)

        def __post_init__(self) -> None:
            self.enable_corruption = False
            self.concatenate_terms = True

    # observation groups
    policy: PolicyCfg = PolicyCfg()

**Reward function** in line 111-136

In [ ]:
@configclass
class RewardsCfg:
    """Reward terms for the MDP."""

    # (1) Constant running reward
    alive = RewTerm(func=mdp.is_alive, weight=1.0)
    # (2) Failure penalty
    terminating = RewTerm(func=mdp.is_terminated, weight=-2.0)
    # (3) Primary task: keep pole upright
    pole_pos = RewTerm(
        func=mdp.joint_pos_target_l2,
        weight=-1.0,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"]), "target": 0.0},
    )
    # (4) Shaping tasks: lower cart velocity
    cart_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.01,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"])},
    )
    # (5) Shaping tasks: lower pole angular velocity
    pole_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.005,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"])},
    )



**Termination condition** in line 139-149

In [ ]:
@configclass
class TerminationsCfg:
    """Termination terms for the MDP."""

    # (1) Time out
    time_out = DoneTerm(func=mdp.time_out, time_out=True)
    # (2) Cart out of bounds
    cart_out_of_bounds = DoneTerm(
        func=mdp.joint_pos_out_of_manual_limit,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"]), "bounds": (-3.0, 3.0)},
    )


#### 2. What are the action space and observation space for an agent defined in the Isaac-Cartpole-v0 task?

Action space is action controls the torque applied to the joint which moves the cart horizontally.

In [ ]:
@configclass
class ActionsCfg:
    """Action specifications for the MDP."""

    joint_effort = mdp.JointEffortActionCfg(asset_name="robot", joint_names=["slider_to_cart"], scale=100.0)

Observation space includes:
1. Relative Position (joint_pos_rel):
- Cart Position: Horizontal position of the cart.
- Pole Angle: Angular position of the pole relative to the vertical.
2. Relative Velocity (joint_vel_rel):
- Cart Velocity: Horizontal velocity of the cart.
- Pole Angular Velocity: Angular velocity of the pole.

In [ ]:
@configclass
class ObservationsCfg:
    """Observation specifications for the MDP."""

    @configclass
    class PolicyCfg(ObsGroup):
        """Observations for policy group."""

        # observation terms (order preserved)
        joint_pos_rel = ObsTerm(func=mdp.joint_pos_rel)
        joint_vel_rel = ObsTerm(func=mdp.joint_vel_rel)

        def __post_init__(self) -> None:
            self.enable_corruption = False
            self.concatenate_terms = True

    # observation groups
    policy: PolicyCfg = PolicyCfg()

#### 3. How can episodes in the Isaac-Cartpole-v0 task be terminated?

In the Isaac-Cartpole-v0 task, episodes can be terminated in two ways as

1. Time out: The episode terminates when a predefined time limit is reached.

In [ ]:
time_out = DoneTerm(func=mdp.time_out, time_out=True)

2. Cart out of bounds: The episode terminates if the cart moves beyond specified position limits, which checks whether the cart's position lies outside the bounds (-3.0, 3.0)


In [ ]:
cart_out_of_bounds = DoneTerm(
        func=mdp.joint_pos_out_of_manual_limit,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"]), "bounds": (-3.0, 3.0)},
    )


#### 4. How many reward terms are used to train an agent in the Isaac-Cartpole-v0 task?

There are 5 reward terms line 111-136

1. Constant running reward เป็น Reward (+1) ที่ให้เมื่อ Agent ยังสามารถมีชีวิตรอดได้ในแต่ละ Step เพื่อสนับสนุนให้ Agent พยายามอยู่รอดในเกมให้ได้นานที่สุด


In [ ]:
alive = RewTerm(func=mdp.is_alive, weight=1.0)

2. Failure penalty เป็น Reward ลดแต้ม (-2) เมื่อ Cartpole ออกนอกขอบเขต เพื่อกระตุ้นให้ Agent หลีกเลี่ยงการเข้าสู่สถานะล้มเหลว

In [ ]:
 terminating = RewTerm(func=mdp.is_terminated, weight=-2.0)

3. Primary task: keep pole upright เป็นการกระตุ้นให้เสาตั้งตรง หากเสามีการเอียงจากตำแหน่งตั้งตรง (Target Position) จะได้บทลงโทษ เป็น Reward ติดลบมากขึ้นตามระยะห่างระหว่าง Pole Position กับ Target Position


In [ ]:
pole_pos = RewTerm(
        func=mdp.joint_pos_target_l2,
        weight=-1.0,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"]), "target": 0.0},
    )


4. Shaping tasks: lower cart velocity เป็น Reward ที่สนับสนุนให้ Agent ลด Cart Velocity เพื่อเพิ่มความเสถียรให้กับระบบ ให้ Agent เรียนรู้ที่จะเคลื่อนที่ด้วยความเร็วที่เหมาะสม

In [ ]:
cart_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.01,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["slider_to_cart"])},
    )

5. Shaping tasks: lower pole angular velocity เป็น Reward ที่ลด Pole Angular Velocity เพื่อเพิ่มความเสถียรให้กับระบบ ป้องกันไม่ให้เสาแกว่งมากเกินไปจนเสียสมดุล

In [ ]:
    pole_vel = RewTerm(
        func=mdp.joint_vel_l1,
        weight=-0.005,
        params={"asset_cfg": SceneEntityCfg("robot", joint_names=["cart_to_pole"])},
    )


# **Part 2: Playing with Cartpole Rl Agent**

Let us adjust the weight of each reward term specified in the Isaac-Cartpole-v0 task and train the agent. Which results will be affected by this adjustment, and why? Submit the answers.

You may further explore by modifying other aspects, such as the agent's action space, observation space, and termination conditions.

# **Part 3: Mapping RL Fundamentals**

#### 1. What is reinforcement learning and its components according to your understanding? Giving examples of each component according to the diagram consider the Cartpole problem.

Reinforcement Learning คือ แนวทางการเรียนรู้ด้วยการที่ Agent จะทำ Action ใดก็ได้เพื่อให้ได้ Reward มากที่สุด เพื่อเพิ่ม Value Function ให้ได้มากที่สุด และในระหว่างทางก็จะมีการเรียนรู้เกิดขึ้นว่า Action ไหนดีหรือไม่ดี

**1. Agent:** ตัวแทนที่จะคอยเรียนรู้และตัดสินใจ ดูว่าใน State นี้ ควรจะมี Action อย่างไร
- Cartpole problem: การเรียนรู้ที่จะทำให้ Pole ตั้งตรง ไม่เอนเอียงไปมา

**2. Environment:** ระบบที่ Agent ต้องมีการตอบโต้ด้วย
- Cartpole problem: การจำลองฟิสิกส์สำหรับการเคลื่อนที่ของ Cartpole

**3. State:** สถานการณ์ในเวลาหนึ่ง
- Cartpole problem: Torque (ไม่ชัวร์), Cart Position, Pole Angle, Cart Velocity, Pole Angular Velocity

**4. Reward:** สิ่งที่ใช้ในการประเมินผลลัพธ์ที่จะเกิดจากการกระทำของ Agent ว่าดีหรือไม่ดี เพื่อบอกว่า Agent ควรจะเลือกทำ Action ไหน
- Cartpole problem:
    - Reward +1 เมื่อ Agent ยังมีชีวิตรอดอยู่
    - Reward -2 เมื่อ Cartpole ออกนอกขอบเขตแล้ว Agent เข้าสู่สถานะล้มเหลว
    - โดนลด Reward ตามระยะห่างระหว่าง Pole Position กับ Target Position
    - โดนลด Reward หาก Cart Velocity ไม่เหมาะสม
    - โดนลด Reward หาก Pole Angular Velocity ไม่เหมาะสม

**5. Action:** การกระทำ
- Cartpole problem: การกำหนด Torque, Cart Position, Pole Angle, Cart Velocity, Pole Angular Velocity

**6. Policy:** หลักการที่ Agent ใช้ในการตัดสินใจหลังจากวิเคราะห์ความเป็นไปได้ที่เกิดจากการได้รับ Reward หลังทำ Action ก่อนหน้า
- Cartpole problem: ฟังก์ชันที่จับคู่ State กับความน่าจะเป็นของ Action (เช่น ซ้ายหรือขวา)

**7. Value Function:** การประเมิน Reward สะสมที่คาดหวังของ State ปัจจุบัน หลังจากทำตาม Policy
- Cartpole problem: การคาดการณ์รางวัลสะสมจาก Reward ที่ได้รับในแต่ละ case

**8. Model:**
- Cartpole problem:

**9 History:** ข้อมูลในอดีตที่ Agent เก็บไว้เพื่อการเรียนรู้ เช่น การเก็บลำดับของ State, Action, และ Reward (คิดว่าไม่ใช่แต่แปะไว้ก่องงง)
- Cartpole problem:

#### 2. What is the difference between the reward, return, and the value function?

**Reward:** ผลตอบแทนที่ได้รับทันทีหลังจากมี Action ใน State ปัจจุบัน

**Return:** ผลตอบแทนสะสมในอนาคต ที่เริ่มต้นจากเวลาปัจจุบัน(t)

**Value Function:** การคาดการณ์ Return เฉลี่ยในอนาคตจาก State หนึ่ง โดยขึ้นอยู่กับ Policy ที่ใช้อยู่

#### 3. Consider policy, state, value function, and model as mathematical functions, what would each one take as input and output?

**Policy**
- input: State
- output: ความน่าจะเป็นของ Action

**State**
- input: -
- output: การอธิบายลักษณะของ Environment ในเวลานั้น เช่น ความเร็ว

**Value Function**
- input: State
- output: Reward สะสมที่คาดหวังเมื่อเริ่มต้นจาก State นั้นและปฏิบัติตาม Policy

**Model**
- input: State และ Action
- output: ความน่าจะเป็นของการเปลี่ยนแปลงไปยัง State ถัดไป